# py2neo4j

1.创建节点和关系，同时输出节点间存在多个关系的节点（便于后续更改）

2.查询某节点的所有邻接点

3.查询所有以该节点为头节点的子图

4.查询所有以该节点为尾节点的子图

5.查询任意两个节点之间的通路

6.查询与某节点的所有子图（？）

### 1.创建节点和关系,输出节点间存在多个关系的节点

In [1]:
import json
from ast import literal_eval
from py2neo import Node,Relationship,Graph,NodeMatcher,RelationshipMatcher

In [2]:
#读入json
json_info=open('226_test1.json','r')
per_article=literal_eval(json_info.read())

In [3]:
#连接数据库
#graph=Graph('http://localhost:7474/',auth=('neo4j','Zhen010809'))#correct
graph=Graph('bolt://3.93.220.202:7687',auth=('neo4j','choices-laundry-directives'))

In [4]:
#函数定义
def MatchNode(graph,label,attrs):
    '''判断图中是否存在该节点'''
    node_matcher=NodeMatcher(graph)
    #re_value=node_matcher.match(label).where("_.name="+"\'"+attrs+"\'").first()#转义字符，这样写的话，实体中的引号需要改成中文格式的
    re_value=node_matcher.match(label,name=attrs).first()                       #这种写法ok了
    return re_value                                                             #若存在该节点，则返回符合条件的第一个节点；若不存在，则返回None

def CreateNode(graph,label,attrs):
    '''创建节点'''
    re_value=MatchNode(graph,label,attrs)
    if re_value is None:
        #若图中不存在相同标签的节点，则创建并返回新建的节点
        node=Node(label,name=attrs)
        graph.create(node)
        return node
    else: 
        #若存在相同标签的节点，则返回该已有的节点
        return re_value

def CreateRelationship(graph,node1,relation,node2):
    '''创建关系（注意：Relationship()可以自动去重）'''
    r=Relationship(node1,relation,node2)
    return graph.create(r)

In [5]:
#创建节点和关系
for per_sentence in per_article:
    for per_relationship in per_sentence['relation_list']:
        predicate_list=per_relationship['predicate'].split('/')              #分割predicate
        node1=CreateNode(graph,predicate_list[0],per_relationship['subject'])#创建头节点
        node2=CreateNode(graph,predicate_list[1],per_relationship['object']) #创建尾节点
        CreateRelationship(graph,node1,predicate_list[2],node2)              #创建关系

In [45]:
#输出两节点间多个关系的节点
#walk(relationship) 返回一个生成器包含起始node、关系本身、终止node
relationship_list=[]
number=0
for per_sentence in per_article:
    for per_relationship in per_sentence['relation_list']:
        #print(per_relationship)
        temp=per_relationship
        temp['num']=number#number:第几句，编号从0开始
        relationship_list.append(temp)
    number=number+1
#print(relationship_list)

error_relationship_list=[]
number=1
#now=0
for i in relationship_list:
    subject1=i['subject']
    object1=i['object'] 
    #print(relationship_list.index(i))
    #for j in relationship_list[now+1:]:
    for j in relationship_list:
        if i!=j:
            subject2=j['subject']
            object2=j['object']
            if subject1==object2 and object1==subject2:
                print(number)
                print("句子1：",i['num'],per_article[i['num']]['text'])
                print("关系：",subject1,"----",i["predicate"],"--->",object1,'\n')
                print("句子2：",j['num'],per_article[j['num']]['text'])
                print("关系：",subject2,"----",i["predicate"],"--->",object2)
                print("=====================================================================")
                number=number+1

1
句子1： 19 Adding refractory elements W, Mo, Ta, Re and Ru are expected to slow down the diffusion process of the elements in alloys.
关系： Ru ---- Composition/Composition/instance_of ---> refractory elements 

句子2： 55 Compared to the content of elements (W + Mo + Ta) in the Re / Ru-free alloy, the content of elements (W + Mo + Ta + Re + Ru) in the 4.5%Re / 3%Ru containing alloy increases from 17.84 wt% to 22.80 wt%, which indicates that the Ru may enhance the content of the refractory elements in alloy to a great extent.
关系： refractory elements ---- Composition/Composition/instance_of ---> Ru
2
句子1： 23 The double line contrast is attributed to the decomposition of dislocation D on the (100) plane, and the anti-phase boundary (APB) was located in between the double lines of dislocations.
关系： dislocation ---- Structure/Structure/condition_on ---> (100) plane 

句子2： 115 Therefore, the slipping plane of dislocation C is determined as the (100) plane according to bC × μC, which is identified 

### 2.查询某节点的所有邻接点

In [29]:
#def allnode_query():
n=input("请输入要查询的节点名：")

node_matcher=NodeMatcher(graph)
node=node_matcher.match(name=n).first() 

relationship_matcher=RelationshipMatcher(graph)
allnode_relationship=relationship_matcher.match(nodes={node},r_type=None)
print(list(allnode_relationship))
#endnode_query()

请输入要查询的节点名：Ru
[property_of(Node('Property', name='dissolving'), Node('Composition', name='Ru')), cause_effect(Node('Composition', name='Ru'), Node('Property', name='creep resistance')), composition_of(Node('Composition', name='Ru'), Node('Structure', name='gamma’-Ni3Al phase'))]


### 3.查询以该节点为头节点的子图

In [31]:
#def headnode_query():
n=input("请输入要查询的节点名：")

node_matcher=NodeMatcher(graph)
node=node_matcher.match(name=n).first() 

relationship_matcher=RelationshipMatcher(graph)
headnode_relationship=relationship_matcher.match(nodes=[node],r_type=None)
print(list(headnode_relationship))
#headnode_query()

请输入要查询的节点名：Ru
[cause_effect(Node('Composition', name='Ru'), Node('Property', name='creep resistance')), composition_of(Node('Composition', name='Ru'), Node('Structure', name='gamma’-Ni3Al phase'))]


### 4.查询以该节点为尾节点的子图

In [34]:
n=input("请输入要查询的节点名：")

endnode_relationship=set(allnode_relationship).difference(set(headnode_relationship))
list(endnode_relationship)

请输入要查询的节点名：Ru


[property_of(Node('Property', name='dissolving'), Node('Composition', name='Ru'))]

### 5.查询任意两个节点之间的通路

In [36]:
#Path()？
node_matcher=NodeMatcher(graph)
node1=node_matcher.match(name='crossslipping').first() 
node2=node_matcher.match(name='{100} planes').first()

relationship_matcher=RelationshipMatcher(graph)
road=relationship_matcher.match(nodes=(node1,node2),r_type=None)
print(list(road))

[]
